DATA TRANSFORMATION PART 2

Here I combine all the files made in part one to one huge dataset that can be used for modeling

In [ ]:
import pandas as pd
import numpy as np

Since the relationship between the datasets is complex (one team can play with another on more than one ocassion), I wrote some functions to help me identify that

In [1]:
def gap_team(team, oppsquad):
    """a function that detects a gap between a team and a fixed oponent
    returns the indexes of the beginning and the end of the gap"""
    coordinates = df.where(df == oppsquad).stack().index.tolist()
    for i in range(len(coordinates)-1):
        if coordinates[i][0] + 1 != coordinates[i+1][0]:
            #print("nieciągłość po elemencie", coordinates[i][0])
            boundry_low = coordinates[i][0]
            boundry_upper = coordinates[i+1][0]
    q = coordinates[0][0]
    w = coordinates[-1][0]
    return([q, boundry_low+1, boundry_upper, w+1])

def gap_oppsquad(oppsquad, team):
    """a function that detects a gap between a team and a fixed oponent
    returns the indexes of the beginning and the end of the gap"""
    coordinates = df_opp.where(df_opp == team).stack().index.tolist()
    start = coordinates[0][0]
    for i in range(len(coordinates)-1):
        if coordinates[i][0] + 1 != coordinates[i+1][0]:
            #print("nieciągłość po elemencie", coordinates[i][0])
            boundry_low = coordinates[i][0]
            boundry_upper = coordinates[i+1][0]
    q = coordinates[0][0]
    w = coordinates[-1][0]
    return([q, boundry_low+1, boundry_upper, w+1])

def upper_bound_df (team, oppsquad):
    """detects the the ending of the second match with another team
    returns the index of the last match"""
    coordinates = df.where(df == oppsquad).stack().index.tolist()
    list = []
    for i in range(len(coordinates)):
        x = coordinates[i][0]
        list.append(x)
    return(max(list))

def lower_bound_df (team, oppsquad):
    """detects the the ending of the second match with another team, esentially the same as the above but for the oponents
    returns the index of the beginning of the first match"""
    coordinates = df.where(df == oppsquad).stack().index.tolist()
    list = []
    for i in range(len(coordinates)):
        x = coordinates[i][0]
        list.append(x)
    return(min(list))

def lower_bound_df_opp (oppsquad, team):
    """detects the the beginnning of the first match with another team
    returns the"""
    coordinates = df_opp.where(df_opp == team).stack().index.tolist()
    list = []
    for i in range(len(coordinates)):
        x = coordinates[i][0]
        list.append(x)
    return(min(list))

def upper_bound_df_opp (oppsquad, team):
    """detects the the beginnning of the first match with another team
    returns the index of the last game of the second match"""
    coordinates = df_opp.where(df_opp == team).stack().index.tolist()
    list = []
    for i in range(len(coordinates)):
        x = coordinates[i][0]
        list.append(x)
    return(max(list))

def is_gap_team(team, oppsquad):
    """a function that detects if the team played with another team on two seperate ocassions
    returns True if there is a gap and False if there is not"""
    coordinates = df.where(df == oppsquad).stack().index.tolist()
    for i in range(len(coordinates)-1):
            if coordinates[i][0] + 1 != coordinates[i+1][0]:
                return(True)

Now the code that sorts...

In [3]:
raw_data = pd.read_csv("teams data.csv", sep=";")
teams = raw_data.teamname.unique()

wszystko_file = pd.DataFrame()
all_teams_df = pd.DataFrame()
for team in teams:
    df = pd.read_csv(f"mean_stats_{team}.csv", sep=';')
    oppsquads = df.oponents.unique()
    posredni_df = pd.DataFrame()
    for oppsquad in oppsquads:
        df_opp = pd.read_csv(f"mean_stats_{oppsquad}.csv", sep=';')
        if is_gap_team(team, oppsquad) == True:
            #print("team:", team, "oppsqad: ", oppsquad, "THERE IS A GAP")
            marker_list_team = gap_team(team, oppsquad)
            marker_list_oppsquad = gap_oppsquad(oppsquad, team)
            licznik_c = 0
            list = []
            for i in range(marker_list_team[0], marker_list_team[1]):
                diff = df.iloc[i, :-3] - df_opp.iloc[marker_list_oppsquad[0] + licznik_c, :-3]
                licznik_c += 1
                list.append(diff)
            licznik_c_2 = 0
            for i in range(marker_list_team[2], marker_list_team[3]):
                diff = df.iloc[i, :-3] - df_opp.iloc[marker_list_oppsquad[2] + licznik_c_2, :-3]
                licznik_c += 1
                list.append(diff)
            masterfile_df = pd.DataFrame(list)
            results_column = []
            for i in range(marker_list_team[0], marker_list_team[1]):
                results = df.iloc[i, -2]
                results_column.append(results)
            for i in range(marker_list_team[2], marker_list_team[3]):
                results = df.iloc[i, -2]
                results_column.append(results)
            masterfile_df["future result"] = results_column
            masterfile_df = masterfile_df.drop(columns=masterfile_df.columns[0])
            #print(masterfile_df)
            wszystko_file = pd.concat([wszystko_file, masterfile_df])
        else:
            #print("team:", team, "oppsqad: ", oppsquad, "NO GAP")
            a = lower_bound_df(team, oppsquad)
            b = upper_bound_df(team, oppsquad)
            c = lower_bound_df_opp(oppsquad, team)
            d = upper_bound_df_opp(oppsquad, team)
            list = []
            licznik_c = 0
            for i in range(a, b+1):
                diff = df.iloc[i, :-3] - df_opp.iloc[c+licznik_c, :-3]
                licznik_c += 1 
                list.append(diff)
            masterfile_df = pd.DataFrame(list)
            results_column = []
            for i in range(a, b+1):
                results = df.iloc[i, -2]
                results_column.append(results)
            masterfile_df["future result"] = results_column
            masterfile_df = masterfile_df.drop(columns=masterfile_df.columns[0])
            wszystko_file = pd.concat([wszystko_file, masterfile_df])
    posredni_df = pd.concat([posredni_df, wszystko_file])
all_teams_df = pd.concat([all_teams_df, posredni_df])
all_teams_df = all_teams_df.dropna()
all_teams_df = all_teams_df.reset_index(drop=True)
all_teams_df.to_csv("masterfile done.csv")
all_teams_df

,team kpm,dragons,barons,towers,dpm,vspm,earned gpm,monsterkills_pm,cspm,goldat15,xpat15,future result
0,0.076987,0.028571,-0.090476,-0.695238,-134.718128,-0.185134,-27.306101,-0.968975,-1.398766,-872.666667,-665.219048,1.0
1,0.046773,0.000000,-0.200000,-1.266667,-157.622987,-0.214460,-59.177227,-1.279145,-1.704927,-851.666667,-658.733333,0.0
2,-0.003039,-0.025000,-0.295833,-2.016667,-211.525307,-0.393220,-94.854949,-1.253363,-1.796630,-1027.604167,-813.808333,0.0
3,-0.019650,0.000000,-1.000000,2.000000,705.006500,5.300050,93.700150,2.777322,3.850050,154.000000,-1603.500000,0.0
4,0.479667,1.000000,1.000000,6.333333,187.260467,1.957733,441.377333,3.706456,3.117767,1349.333333,-982.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1451,0.004700,-1.500000,-0.500000,-2.500000,302.183200,-0.090650,-32.767300,0.195975,-1.532400,11.500000,1102.500000,0.0
1452,0.701100,5.500000,2.000000,14.500000,1698.845950,5.274850,900.492850,10.928177,9.999650,7614.500000,3171.000000,1.0
1453,-0.035333,-2.333333,-0.333333,-1.333333,-126.338950,-1.799233,-85.683450,-0.548742,-1.591583,273.833333,837.333333,0.0
1454,-0.124610,-3.523810,-0.285714,-0.190476,-556.476652,-1.759019,-121.471481,-0.745499,-2.804424,-222.809524,79.619048,1.0


All done!